<a href="https://colab.research.google.com/github/notwld/chatbot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import nltk
nltk.download("punkt")

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import tensorflow as tf
import numpy as np
import random,json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [126]:
# Using google colab's files method to upload json file which contains the data 

# from google.colab import files
# files.upload()


In [127]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [128]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [129]:
words = []
classes = []
documents = []
ignore = ["?"]

In [130]:
# data preprocessing: looping through json file which is saved in variable 
# intents and tokenizing the patterns, extract word with it's tag and saving 
# them in the list.

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word = nltk.word_tokenize(pattern)
    words.extend(word)

    documents.append((word,intent["tag"]))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



In [131]:
words = [stemmer.stem(word.lower()) for word in words if word not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))


In [132]:
print("Total documents:", len(documents))
print("Total classes:", len(classes))
print("Total unique words:", len(words))

Total documents: 27
Total classes: 8
Total unique words: 52


In [149]:
# creating lists for training the data

training = [] # f(x)
output = [] # f(y)

output_empty = [0] * len(classes)

In [134]:
#test

documents[0]

(['Hi'], 'greeting')

In [150]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


<ipython-input-150-041b20c145fd>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [151]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 



In [152]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/1000
4/4 [==============================] - 1s 7ms/step - loss: 2.1002 - accuracy: 0.1481
Epoch 2/1000
4/4 [==============================] - 0s 4ms/step - loss: 2.0778 - accuracy: 0.1481
Epoch 3/1000
4/4 [==============================] - 0s 7ms/step - loss: 2.0578 - accuracy: 0.1481
Epoch 4/1000
4/4 [==============================] - 0s 4ms/step - loss: 2.0404 - accuracy: 0.1481
Epoch 5/1000
4/4 [==============================] - 0s 4ms/step - loss: 2.0225 - accuracy: 0.2222
Epoch 6/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.0059 - accuracy: 0.2222
Epoch 7/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9880 - accuracy: 0.2222
Epoch 8/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9714 - accuracy: 0.2963
Epoch 9/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9544 - accuracy: 0.3704
Epoch 10/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9375 - accuracy: 0.4074
Epoch 11/1000
4/4 [

In [153]:
# prediction

import pickle
pickle.dump({"words":words,"classes":classes},open("training_data","wb"))



In [154]:
from keras.models import load_model
model = load_model("model.pkl")

In [155]:
with open("training_data","rb") as data:
  data = pickle.load(data)
  words = data["words"]
  classes = data["classes"]


In [156]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [157]:
def cleanUp(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

In [158]:
def bow(sentence, words):
  sentence_words = cleanUp(sentence)
  bag = [0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i] = 1
  bag = np.array(bag)
  return (bag)

In [159]:
ERR_THRESHOLD = 0.30

def classify(sentence):
  bag = bow(sentence,words) # generating all the possibilties from a sentence
  result = model.predict(np.array([bag]))

  result = [[i,r] for i,r in enumerate(result[0])]
  result.sort(key=lambda x: x[1], reverse=True)

  result_li = []
  for res in result:
    if res[1] > ERR_THRESHOLD:
      result_li.append((classes[res[0]],res[1])) #intent and probability

  return result_li




1/1 [==============================] - 0s 91ms/step


[('greeting', 0.99915063)]

In [160]:
def get_response(sentence):
  res = classify(sentence)
  if res:
    while res:
      for intent in intents["intents"]:
        if intent["tag"] == res[0][0]:
          return print(random.choice(intent["responses"])) 

      res.pop(0)


In [168]:
get_response("Who are you?")


1/1 [==============================] - 0s 69ms/step
Thanks for asking. I am Mayank Bajaj, coder by profession but ML enthusiast by passion.
